In [201]:
#!/usr/bin/env python

import sys
sys.path.insert(0,'/Users/cmdb/projects/ConfocalAnalysis')
sys.path.insert(0,'/Users/babygirljohnston/Documents/ConfocalAnalysis/ConfocalAnalysis')
sys.path.insert(0,'/Users/babygirljohnston/Documents/Confocal_Xiuqi/scikit-image')

from czifile import CziFile
import numpy as np
from math import sqrt
from skimage.feature import blob_log
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.cluster import DBSCAN
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D
import operator
import matplotlib.cm as cm

%matplotlib qt 
#interactive mode of plt

#%matplotlib inline 
# plt in line with notebook
'''Global variables'''
colomap=["Blues","gray","Greens_r","Reds_r"] # colormap for channels
labell=['yellow','yellow','yellow','yellow'] # label color for blobs

'''Setup tips'''
# pip install scikit-image # if not working, compile from source
# pip install -U scikit-learn
# If weird errors happen, try update the packages except 'pip install setuptools==33.1.1'
# different mpl might have version clahes, remember to clean the older version.
# I manually replace the /System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/mpl_toolkits
# with the newer version from /Library/Python/2.7/site-packages/mpl_toolkits

"""Useful Functions"""

def GetOneChannel( czi_file, channel ):
# this function takes the whole .czi file and return a single channel .czi
    stacknumber=czi_file.shape[3]
    OneChannel=[czi_file[0,channel,0,index].T[0].T for index in range(stacknumber)]
    return OneChannel

def FindBlobs( OneChannel,thresh=0.2 ):
# this function takes output from GetOneChannel (a single channel .czi file input)
# return all the blobs in a dict & a structured list & a flattened list
# Dict    # "Y,X,Z"->[radius, intensity]
# struct. List    # [[[Y,X,Z1,radius,intensity],[Y,X,Z1,...]...],...,[[Y,X,Zn,...],...]],np.array
# flat. List    # [[Y1,X1,Z1],[Y2,X2,Z2],[Y3,X3,Z3],...,[Yn,X3,Z3]]
  
    blobs_dict={}
    blobsAll=[]
    blobsList=[]
    stacknumber=len(OneChannel)
    
    for z in range(stacknumber):
        planeblobs=[]
        """Blob Recognition Parameters here"""
        log = blob_log(OneChannel[z], max_sigma=30, num_sigma=10,min_sigma =3, threshold=thresh)
        # Radius of the blob
        log[:, 2] = log[:, 2] * sqrt(2)
        # intensity
        blob_intensity=np.zeros(len(log[:,0]))
        
        for i in range(len(log[:,0])):
            blob_intensity[i]=OneChannel[z][int(log[i,0]),int(log[i,1])]
        
        for i in range(len(log[:,2])):
            position=np.append(log[i,0:2],z)
            position=map(int,position)
            blobs_dict["{},{},{}".format(position[0],position[1],position[2])]=[log[i,2],blob_intensity[i]]
            blobsfeature=[log[i,0],log[i,1],z,log[i,2],blob_intensity[i]]
            print blobsfeature
            planeblobs.append(blobsfeature)
            blobsList.append(position)
        
        blobsAll.append(planeblobs)
    
    blobsAll=np.array(blobsAll)
    blobsList=np.array(blobsList)
    
    return blobs_dict,blobsAll,blobsList



def GetClustersOneChannel( blobsList ):
# this function takes the dict and flattened list from FindBlobs
# return the clusters
###  [[[cluster1_dot1],[cluster1_dot2],...],
#     [[cluster2_dot1],[cluster2_dot2]...],
#     [...],...,
#     [[clusterN_dot1],[clusterN_dot2]] ]

    """3D clustering of Blobs"""
    db = DBSCAN(eps=2, min_samples=2).fit(blobsList)  # eps is the max distance between 2 dots in one cluster
    labels = db.labels_
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    print('Estimated number of clusters: %d' % n_clusters_)
#     print("Silhouette Coefficient: %0.3f"
#           % metrics.silhouette_score(blobsList, labels))
    
    """Group up the dots within a cluster"""
    clusters=[]
    for i in range(n_clusters_):
        cluster=[]
        ## Extract X,Y,Z location of the points based on clusters
        positions=np.argwhere(labels==i)
        for j in range(len(positions)):
            cluster.append(blobsList[positions[j]])
        #print cluster
        clusters.append(cluster)
    
    return clusters

def SignalPositionsOneChannel( clusters,blobs_dict ):
# This function takes the clusters List from GetClustersOneChannel
# return the calculated signal position in this one channel all stacks
# [[Y,X,Z,avgIntensity],[Y,X,Z,avgIntensity],...,[Y,X,Z,avgIntensity]]
    """Find the actual signal location"""
    signals=[]
    for i in range(len(clusters)):
        signal=[]
        dots=[]
        ## extract the intensity of the dot in the cluster (from the dictionary)
        for j in range(len(clusters[i])):
            #print clusters[i][j][0]
            dot=clusters[i][j][0]
            intensity=blobs_dict["{},{},{}".format(clusters[i][j][0][0],clusters[i][j][0][1],clusters[i][j][0][2])][1]
            dot=np.append(dot,intensity)
            dots.append(dot)
        # Calculating the actual dot location
        intensity_sum=sum(dots[index][3] for index in range(len(dots)))
        avgIntensity=intensity_sum/len(dots)
        y=sum(dots[index][0]*dots[index][3]/intensity_sum for index in range(len(dots)))
        x=sum(dots[index][1]*dots[index][3]/intensity_sum for index in range(len(dots)))
        z=sum(dots[index][2]*dots[index][3]/intensity_sum for index in range(len(dots)))
        signal=[y,x,z,avgIntensity]
        signals.append(signal)
    signals=np.array(signals)
    return signals

def ToActualScale(list):
# This function takes a [n_sample,[Y,X,Z,Intensity]] LIST
# convert to the actual scale of the coordinates
    """Adjust Parameters"""
    # 63X objective, unit: um
    x_scale=0.099
    y_scale=0.099
    z_scale=0.2
    actual=np.zeros((len(list),len(list[0])))
    for i in range(len(list)):
        actual[i][0]=list[i][0]*y_scale
        actual[i][1]=list[i][1]*x_scale
        actual[i][2]=list[i][2]*z_scale
        if len(list[0])==4:
            actual[i][3]=list[i][3]
    return actual

def GetDistance(element1,element2):
# This function takes two [Y,X,Z]
# return euclidean distance
    distance=sqrt((element1[0]-element2[0])**2+(element1[1]-element2[1])**2+(element1[2]-element2[2])**2)
    return distance

def FindPairs( signals1,signals2,limit=1.2 ):
# This function takes 2 signals-Lists generated in SignalPositionsOneChannel
# return pairs  (LIST)
# [[sig1_1,sig2_N1,dist],[sig1_2,sig2_N2,dist],...,[sig1_N,sig2_Nn,dist]]
    pairs=[]
    signals1=ToActualScale(signals1)
    signals2=ToActualScale(signals2)
    #print signals1
    #print signals2
    
    # Brute-Force calculate the distance matrix between the two signal groups
    # 
    row_num= len(signals1)
    col_num= len(signals2)
    matrix=np.zeros((row_num,col_num))
    for i in range(row_num):
        for j in range(col_num):
            matrix[i][j]=GetDistance(signals1[i],signals2[j])
    while 1:
        min_idx=matrix.argmin()
        row=min_idx/col_num
        col=min_idx%col_num
        distance=matrix[row][col]
        if distance > limit:
            break
        pairs.append([row,col,distance])
        matrix[row,:]=100
        matrix[:,col]=100
    return pairs


def PlotOriginal(image_arrays,channel,stack):
    # This function takes the czifile from the reader line, specific channel, stack and colormap
    # plot the image
    images=GetOneChannel(image_arrays,channel)
    fig=plt.figure(figsize=(20,20))
    mapp=plt.imshow(images[stack],cmap=colomap[channel])
    cbar = fig.colorbar(mapp, ticks=[0, 65535])
    cbar.ax.set_yticklabels(['0', '65535'])
    plt.title("Channel {}, Stack {}".format(channel,stack),fontsize=20)
    return mapp

def PlotCheckBlobs(OneChannel, blobsAll, channel, stack):
    # Take Single channel all stack, blobsAll from FindBlobs, stack,label
    # Plot the specific image on that stack, together with the blobs.
    fig, axes = plt.subplots(1, 1, sharex=True, sharey=True,figsize=(20,20))
    mapp=plt.imshow(OneChannel[stack],cmap=colomap[channel])

    cbar = fig.colorbar(mapp, ticks=[0, 65535])
    cbar.ax.set_yticklabels(['0', '65535'])

    log = blobsAll[stack]
    #print log
    for blob in log:
        #print blob
        y, x, z,radius,intensity = blob
        c = plt.Circle((x, y), radius, color=labell[channel], linewidth=2, fill=False)   
        axes.add_patch(c)
    
    plt.title("Channel {}, Stack {}".format(channel,stack),fontsize=20)
    
    return mapp


def PlotCheckClusters(clusters):
    # This function takes the clusters & plot each cluster in a different color to inspect
    colors = cm.rainbow(np.linspace(0, 1, len(clusters_ss)))
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111, projection='3d')
    for cluster, c in zip(clusters_ss, colors):
        #print cluster
        y,x,z=np.array([cluster[index] for index in range(len(cluster))]).T
        ax.scatter(x, y, z,color=c)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    ax.set_zlim([-2,20])
    return 0

def PlotPairs(pair_coord, threshold=0):
    # This function takes the actual pair_coord, same structure of FindPairs Output
    # takes the min threshold and plot the pairs above the threshold in 3D
    
    long=[]
    for pair in pairs:
        #print pair
        if pair[2]>threshold:
            long.append(pair[0:3])

    pair_coord=[]
    for pair in long:
        pair_coord.append([signals_ss[pair[0]][0:3],signals_klu[pair[1]][0:3]])

    y_s,x_s,z_s = np.array([pair_coord[index][0] for index in range(len(pair_coord))]).T
    y_k,x_k,z_k = np.array([pair_coord[index][1] for index in range(len(pair_coord))]).T
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_s, y_s, z_s,color='green')
    ax.scatter(x_k, y_k, z_k,color='red')
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    ax.set_zlim([-5,30])
    plt.title("3D Plot of Signal Pairs with Distance above {} um".format(threshold),fontsize=20)
    return ax
    

In [9]:
""" Image Input """
with CziFile('/Users/babygirljohnston/Documents/Confocal_Xiuqi/Image_3.czi') as czi:
    image_arrays=czi.asarray()
# 63X objective, unit: um

In [192]:
## Plot the original image of any channel, any stack

channel=3
stack=7

PlotOriginal(image_arrays,channel,stack)
# plt.plot((x1, x2), (y1, y2), 'k-') #to draw a line from the point (x1, y1) to the point (x2, y2)

In [12]:
"""Image Processing -- (time consuming: several minutes)""" 
# Find blobs in one channel image image for ss
OneChannel_ss=GetOneChannel(image_arrays,2)
blobs_dict_ss,blobsAll_ss,blobsList_ss=FindBlobs(OneChannel_ss,0.1)

# Find blobs in one channel image stack for klu
OneChannel_klu=GetOneChannel(image_arrays,3)
blobs_dict_klu,blobsAll_klu,blobsList_klu=FindBlobs(OneChannel_klu,0.1)

[995.0, 812.0, 0, 4.2426406871192857, 14714.0]
[986.0, 1023.0, 0, 4.2426406871192857, 22138.0]
[986.0, 776.0, 0, 4.2426406871192857, 34190.0]
[986.0, 700.0, 0, 4.2426406871192857, 42184.0]
[979.0, 540.0, 0, 4.2426406871192857, 21615.0]
[973.0, 79.0, 0, 4.2426406871192857, 62062.0]
[968.0, 716.0, 0, 4.2426406871192857, 23964.0]
[964.0, 470.0, 0, 4.2426406871192857, 55880.0]
[948.0, 731.0, 0, 4.2426406871192857, 29353.0]
[945.0, 850.0, 0, 4.2426406871192857, 28910.0]
[945.0, 726.0, 0, 4.2426406871192857, 31280.0]
[936.0, 408.0, 0, 4.2426406871192857, 17950.0]
[906.0, 568.0, 0, 4.2426406871192857, 19749.0]
[904.0, 855.0, 0, 4.2426406871192857, 21678.0]
[893.0, 808.0, 0, 4.2426406871192857, 23384.0]
[887.0, 5.0, 0, 4.2426406871192857, 16619.0]
[885.0, 684.0, 0, 4.2426406871192857, 32235.0]
[872.0, 498.0, 0, 4.2426406871192857, 21192.0]
[868.0, 136.0, 0, 4.2426406871192857, 27093.0]
[866.0, 639.0, 0, 4.2426406871192857, 39885.0]
[864.0, 751.0, 0, 4.2426406871192857, 18740.0]
[863.0, 623.0, 

[995.0, 811.0, 1, 4.2426406871192857, 29923.0]
[990.0, 444.0, 1, 4.2426406871192857, 13812.0]
[986.0, 701.0, 1, 4.2426406871192857, 28671.0]
[985.0, 1023.0, 1, 4.2426406871192857, 39905.0]
[985.0, 777.0, 1, 4.2426406871192857, 33536.0]
[979.0, 539.0, 1, 4.2426406871192857, 13127.0]
[973.0, 78.0, 1, 4.2426406871192857, 65535.0]
[971.0, 356.0, 1, 4.2426406871192857, 26110.0]
[968.0, 716.0, 1, 4.2426406871192857, 20534.0]
[966.0, 427.0, 1, 4.2426406871192857, 19578.0]
[963.0, 470.0, 1, 4.2426406871192857, 53583.0]
[947.0, 730.0, 1, 4.2426406871192857, 29470.0]
[946.0, 1023.0, 1, 4.2426406871192857, 25750.0]
[945.0, 850.0, 1, 4.2426406871192857, 13417.0]
[945.0, 726.0, 1, 4.2426406871192857, 27177.0]
[935.0, 409.0, 1, 4.2426406871192857, 31632.0]
[930.0, 587.0, 1, 4.2426406871192857, 20918.0]
[912.0, 50.0, 1, 4.2426406871192857, 12272.0]
[907.0, 372.0, 1, 4.2426406871192857, 22223.0]
[905.0, 568.0, 1, 4.2426406871192857, 23642.0]
[892.0, 808.0, 1, 4.2426406871192857, 31154.0]
[891.0, 970.0

[1007.0, 454.0, 2, 4.2426406871192857, 28470.0]
[995.0, 812.0, 2, 4.2426406871192857, 48193.0]
[986.0, 701.0, 2, 4.2426406871192857, 24616.0]
[985.0, 1023.0, 2, 4.2426406871192857, 32725.0]
[985.0, 777.0, 2, 4.2426406871192857, 31958.0]
[973.0, 78.0, 2, 4.2426406871192857, 58985.0]
[970.0, 585.0, 2, 4.2426406871192857, 19509.0]
[970.0, 357.0, 2, 4.2426406871192857, 39746.0]
[963.0, 470.0, 2, 4.2426406871192857, 29789.0]
[947.0, 731.0, 2, 4.2426406871192857, 32026.0]
[946.0, 1023.0, 2, 4.2426406871192857, 31240.0]
[945.0, 727.0, 2, 4.2426406871192857, 20306.0]
[935.0, 409.0, 2, 4.2426406871192857, 19328.0]
[930.0, 987.0, 2, 4.2426406871192857, 24023.0]
[930.0, 587.0, 2, 4.2426406871192857, 32482.0]
[925.0, 448.0, 2, 4.2426406871192857, 17930.0]
[916.0, 510.0, 2, 4.2426406871192857, 17394.0]
[912.0, 50.0, 2, 4.2426406871192857, 23768.0]
[896.0, 354.0, 2, 4.2426406871192857, 27281.0]
[892.0, 808.0, 2, 4.2426406871192857, 40916.0]
[890.0, 971.0, 2, 4.2426406871192857, 18332.0]
[885.0, 235.

[1007.0, 454.0, 3, 4.2426406871192857, 25452.0]
[994.0, 813.0, 3, 4.2426406871192857, 51113.0]
[988.0, 888.0, 3, 4.2426406871192857, 26342.0]
[985.0, 1023.0, 3, 4.2426406871192857, 30387.0]
[984.0, 777.0, 3, 4.2426406871192857, 38356.0]
[973.0, 79.0, 3, 4.2426406871192857, 46239.0]
[970.0, 585.0, 3, 4.2426406871192857, 32633.0]
[970.0, 357.0, 3, 4.2426406871192857, 42190.0]
[952.0, 979.0, 3, 4.2426406871192857, 19915.0]
[947.0, 732.0, 3, 4.2426406871192857, 33776.0]
[945.0, 1023.0, 3, 4.2426406871192857, 31202.0]
[944.0, 726.0, 3, 4.2426406871192857, 34185.0]
[944.0, 467.0, 3, 4.2426406871192857, 25936.0]
[935.0, 409.0, 3, 4.2426406871192857, 25083.0]
[930.0, 987.0, 3, 4.2426406871192857, 22447.0]
[929.0, 587.0, 3, 4.2426406871192857, 46123.0]
[925.0, 449.0, 3, 4.2426406871192857, 22141.0]
[920.0, 535.0, 3, 4.2426406871192857, 25409.0]
[916.0, 511.0, 3, 4.2426406871192857, 30374.0]
[912.0, 49.0, 3, 4.2426406871192857, 24764.0]
[899.0, 606.0, 3, 4.2426406871192857, 30288.0]
[896.0, 354.

[1007.0, 454.0, 4, 4.2426406871192857, 16649.0]
[994.0, 813.0, 4, 4.2426406871192857, 46619.0]
[988.0, 888.0, 4, 4.2426406871192857, 35544.0]
[985.0, 1023.0, 4, 4.2426406871192857, 24494.0]
[984.0, 777.0, 4, 4.2426406871192857, 11839.0]
[974.0, 76.0, 4, 4.2426406871192857, 18553.0]
[972.0, 650.0, 4, 4.2426406871192857, 34041.0]
[970.0, 358.0, 4, 4.2426406871192857, 43364.0]
[969.0, 585.0, 4, 4.2426406871192857, 29677.0]
[952.0, 979.0, 4, 4.2426406871192857, 13607.0]
[947.0, 436.0, 4, 4.2426406871192857, 20225.0]
[943.0, 468.0, 4, 4.2426406871192857, 42178.0]
[941.0, 843.0, 4, 4.2426406871192857, 22015.0]
[941.0, 307.0, 4, 4.2426406871192857, 29332.0]
[930.0, 987.0, 4, 4.2426406871192857, 24355.0]
[929.0, 587.0, 4, 4.2426406871192857, 43305.0]
[920.0, 535.0, 4, 4.2426406871192857, 29415.0]
[918.0, 30.0, 4, 4.2426406871192857, 20999.0]
[916.0, 511.0, 4, 4.2426406871192857, 25239.0]
[907.0, 55.0, 4, 4.2426406871192857, 29881.0]
[898.0, 606.0, 4, 4.2426406871192857, 40017.0]
[896.0, 353.0,

...

[1012.0, 602.0, 7, 4.2426406871192857, 18758.0]
[968.0, 363.0, 7, 4.2426406871192857, 28888.0]
[933.0, 486.0, 7, 4.2426406871192857, 20040.0]
[931.0, 501.0, 7, 4.2426406871192857, 15951.0]
[925.0, 826.0, 7, 4.2426406871192857, 16601.0]
[924.0, 166.0, 7, 4.2426406871192857, 24724.0]
[908.0, 786.0, 7, 4.2426406871192857, 27660.0]
[899.0, 299.0, 7, 4.2426406871192857, 17637.0]
[895.0, 958.0, 7, 4.2426406871192857, 25552.0]
[886.0, 268.0, 7, 4.2426406871192857, 20292.0]
[881.0, 786.0, 7, 4.2426406871192857, 28978.0]
[877.0, 388.0, 7, 4.2426406871192857, 21625.0]
[876.0, 717.0, 7, 4.2426406871192857, 21307.0]
[864.0, 453.0, 7, 4.2426406871192857, 47041.0]
[854.0, 291.0, 7, 4.2426406871192857, 13312.0]
[852.0, 282.0, 7, 4.2426406871192857, 21876.0]
[848.0, 578.0, 7, 4.2426406871192857, 30137.0]
[844.0, 254.0, 7, 4.2426406871192857, 13280.0]
[841.0, 742.0, 7, 4.2426406871192857, 20160.0]
[838.0, 956.0, 7, 4.2426406871192857, 23579.0]
[834.0, 42.0, 7, 4.2426406871192857, 22393.0]
[830.0, 655.0

[949.0, 934.0, 9, 4.2426406871192857, 29806.0]
[947.0, 56.0, 9, 4.2426406871192857, 20765.0]
[911.0, 249.0, 9, 4.2426406871192857, 30676.0]
[905.0, 102.0, 9, 4.2426406871192857, 18522.0]
[904.0, 788.0, 9, 4.2426406871192857, 20050.0]
[900.0, 225.0, 9, 4.2426406871192857, 19182.0]
[899.0, 299.0, 9, 4.2426406871192857, 27795.0]
[886.0, 209.0, 9, 4.2426406871192857, 12708.0]
[879.0, 497.0, 9, 4.2426406871192857, 20864.0]
[876.0, 717.0, 9, 4.2426406871192857, 31943.0]
[875.0, 458.0, 9, 4.2426406871192857, 40395.0]
[853.0, 282.0, 9, 4.2426406871192857, 27691.0]
[828.0, 439.0, 9, 4.2426406871192857, 16044.0]
[822.0, 179.0, 9, 4.2426406871192857, 34903.0]
[816.0, 751.0, 9, 4.2426406871192857, 46505.0]
[816.0, 182.0, 9, 4.2426406871192857, 39006.0]
[811.0, 677.0, 9, 4.2426406871192857, 14851.0]
[801.0, 685.0, 9, 4.2426406871192857, 47147.0]
[793.0, 113.0, 9, 4.2426406871192857, 21716.0]
[790.0, 831.0, 9, 4.2426406871192857, 22677.0]
[780.0, 464.0, 9, 4.2426406871192857, 21083.0]
[774.0, 135.0,

[1020.0, 532.0, 11, 4.2426406871192857, 41364.0]
[1017.0, 593.0, 11, 4.2426406871192857, 23486.0]
[1002.0, 106.0, 11, 4.2426406871192857, 9801.0]
[954.0, 292.0, 11, 4.2426406871192857, 33069.0]
[925.0, 686.0, 11, 4.2426406871192857, 22644.0]
[916.0, 245.0, 11, 4.2426406871192857, 27715.0]
[904.0, 788.0, 11, 4.2426406871192857, 36197.0]
[897.0, 827.0, 11, 4.2426406871192857, 13409.0]
[891.0, 603.0, 11, 4.2426406871192857, 13214.0]
[887.0, 377.0, 11, 4.2426406871192857, 29024.0]
[885.0, 930.0, 11, 4.2426406871192857, 21418.0]
[863.0, 771.0, 11, 4.2426406871192857, 25249.0]
[860.0, 32.0, 11, 4.2426406871192857, 23149.0]
[857.0, 871.0, 11, 4.2426406871192857, 20565.0]
[857.0, 613.0, 11, 4.2426406871192857, 24134.0]
[851.0, 227.0, 11, 4.2426406871192857, 11680.0]
[844.0, 163.0, 11, 4.2426406871192857, 40298.0]
[840.0, 820.0, 11, 4.2426406871192857, 17953.0]
[834.0, 328.0, 11, 4.2426406871192857, 26489.0]
[833.0, 890.0, 11, 4.2426406871192857, 20462.0]
[820.0, 558.0, 11, 4.2426406871192857, 

[1022.0, 106.0, 13, 4.2426406871192857, 12808.0]
[1018.0, 593.0, 13, 4.2426406871192857, 12339.0]
[996.0, 369.0, 13, 4.2426406871192857, 20334.0]
[990.0, 78.0, 13, 4.2426406871192857, 21071.0]
[987.0, 277.0, 13, 4.2426406871192857, 23864.0]
[981.0, 780.0, 13, 4.2426406871192857, 29447.0]
[968.0, 528.0, 13, 4.2426406871192857, 21198.0]
[917.0, 244.0, 13, 4.2426406871192857, 33627.0]
[916.0, 933.0, 13, 4.2426406871192857, 22100.0]
[896.0, 826.0, 13, 4.2426406871192857, 19535.0]
[893.0, 612.0, 13, 4.2426406871192857, 36403.0]
[891.0, 603.0, 13, 4.2426406871192857, 23960.0]
[888.0, 942.0, 13, 4.2426406871192857, 22151.0]
[877.0, 258.0, 13, 4.2426406871192857, 29158.0]
[861.0, 31.0, 13, 4.2426406871192857, 34379.0]
[854.0, 708.0, 13, 4.2426406871192857, 32207.0]
[845.0, 961.0, 13, 4.2426406871192857, 19772.0]
[836.0, 1011.0, 13, 4.2426406871192857, 24601.0]
[835.0, 168.0, 13, 4.2426406871192857, 23633.0]
[832.0, 891.0, 13, 4.2426406871192857, 19559.0]
[832.0, 375.0, 13, 4.2426406871192857, 

[1023.0, 554.0, 15, 8.4852813742385713, 92.0]
[1023.0, 106.0, 15, 4.2426406871192857, 26924.0]
[1003.0, 772.0, 15, 4.2426406871192857, 20468.0]
[997.0, 369.0, 15, 4.2426406871192857, 14394.0]
[991.0, 77.0, 15, 4.2426406871192857, 34302.0]
[973.0, 336.0, 15, 4.2426406871192857, 41323.0]
[969.0, 529.0, 15, 4.2426406871192857, 17869.0]
[919.0, 697.0, 15, 4.2426406871192857, 28347.0]
[903.0, 441.0, 15, 4.2426406871192857, 20300.0]
[879.0, 589.0, 15, 4.2426406871192857, 13676.0]
[875.0, 865.0, 15, 4.2426406871192857, 23269.0]
[871.0, 552.0, 15, 4.2426406871192857, 18490.0]
[854.0, 708.0, 15, 4.2426406871192857, 33197.0]
[853.0, 929.0, 15, 4.2426406871192857, 18984.0]
[847.0, 735.0, 15, 4.2426406871192857, 21415.0]
[832.0, 374.0, 15, 4.2426406871192857, 26539.0]
[828.0, 477.0, 15, 4.2426406871192857, 22916.0]
[818.0, 688.0, 15, 4.2426406871192857, 16242.0]
[818.0, 35.0, 15, 4.2426406871192857, 18288.0]
[817.0, 581.0, 15, 4.2426406871192857, 33100.0]
[809.0, 738.0, 15, 4.2426406871192857, 232

In [194]:
"""Check the blobs"""

stack=3   # Change stack number to inspect across Z-stack

channel=2 # to indicate colormap for plotting
PlotCheckBlobs(OneChannel_ss, blobsAll_ss,channel, stack,)
channel=3 # to indicate colormap for plotting
PlotCheckBlobs(OneChannel_klu, blobsAll_klu, channel, stack)



In [247]:
"""Cluster the blobs across planes"""
clusters_ss=GetClustersOneChannel(blobsList_ss)
clusters_klu=GetClustersOneChannel(blobsList_klu)

# Plot the clusters in 3D
PlotCheckClusters(clusters_ss)
plt.title("channel 2",fontsize=20)

PlotCheckClusters(clusters_klu)
plt.title("Channel 3",fontsize=20)
    

Estimated number of clusters: 1394
Estimated number of clusters: 825


In [43]:
'''Find the actual position of the signals'''
signals_ss=SignalPositionsOneChannel(clusters_ss,blobs_dict_ss)
signals_klu=SignalPositionsOneChannel(clusters_klu,blobs_dict_klu)


In [199]:
'''plot the two signals'''
y_s,x_s,z_s,i_s = ToActualScale(signals_ss).T
y_k,x_k,z_k,i_k = ToActualScale(signals_klu).T

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x_s, y_s, z_s,color='green')
ax.scatter(x_k, y_k, z_k,color='red')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_zlim([-1,6])

plt.title("Actual Signal Positions", fontsize=20)
plt.show()

In [166]:
'''Find the pairs between the two channels'''
# empirically, distance between 2 dots won't exceed 1.2 microns
'''Adjust Max pair distance here'''
pairs=FindPairs(signals_ss,signals_klu,1.4)

print "Found {} signal pairs".format(len(pairs))

# print pairs

Found 489 signal pairs


In [202]:
'''Plot all the pairs in 3D'''
pair_coord=[]
for pair in pairs:
    pair_coord.append([signals_ss[pair[0]][0:3],signals_klu[pair[1]][0:3]])

for i in range(len(pair_coord)):
     pair_coord[i]=ToActualScale(pair_coord[i])
PlotPairs(pair_coord)

# print pair_coord

In [207]:
'''extract the long pairs'''

PlotPairs(pair_coord,1.2)

In [241]:
'''calculate the distance distribution of the pairs'''
distances=[pairs[index][2] for index in range(len(pairs))]
hist, bin_edges=np.histogram(distances,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(0, 1.4)
plt.show()


In [246]:
'''Check pair orientation'''

y=np.array([pair_coord[index][0][0]-pair_coord[index][1][0] for index in range(len(pair_coord))]).T
x=np.array([pair_coord[index][0][1]-pair_coord[index][1][1] for index in range(len(pair_coord))]).T
z=np.array([pair_coord[index][0][2]-pair_coord[index][1][2] for index in range(len(pair_coord))]).T

plt.subplot(131)
hist, bin_edges=np.histogram(y,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(-1.5, 1.5)
plt.axvline(0, color='k')
plt.title("Y axis distance distribution")
plt.show()

plt.subplot(132)
hist, bin_edges=np.histogram(x,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(-1.5, 1.5)
plt.title("X axis distance distribution")
plt.axvline(0, color='k')
plt.show()

plt.subplot(133)
hist, bin_edges=np.histogram(z,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(-1.5, 1.5)
plt.title("Z axis distance distribution")
plt.axvline(0, color='k')
plt.show()



In [232]:
'''plot long pairs with image'''
## Extract Channel 1 -- Z-stack
channel=1
stack=7
# lamin channel
OneChannel_lamin=GetOneChannel(image_arrays,1)

# Show the lamin channel
fig, axes = plt.subplots(1, 1, sharex=True, sharey=True,figsize=(20,20))
mapp=plt.imshow(OneChannel_lamin[stack],cmap=colomap[channel])

cbar = fig.colorbar(mapp, ticks=[0, 65535])
cbar.ax.set_yticklabels(['0', '65535'])

long=[]
for pair in pairs:
    #print pair
    if pair[2]>1.1:
        long.append(pair[0:3])

# plot all the long pairs
for pair in long:
    ss, klu, distance= pair
    s = plt.Circle((signals_ss[ss][1], signals_ss[ss][0]), 2, color=labell[2], linewidth=2, fill=False)
    axes.add_patch(s)
    k = plt.Circle((signals_klu[klu][1], signals_klu[klu][0]), 2, color=labell[3], linewidth=2, fill=False)
    axes.add_patch(k)
plt.show()


In [235]:
'''Check distribution of pairs'''
#print long
y=np.array([long[index][0] for index in range(len(long))]).T
x=np.array([long[index][1] for index in range(len(long))]).T
z=np.array([long[index][2] for index in range(len(long))]).T
#print y

plt.subplot(131)
hist, bin_edges=np.histogram(y,bins=10)
plt.bar(bin_edges[:-1],hist,width=100)
plt.xlim(0,1024)
plt.title("Y Axis Position Distribution")
plt.show()

plt.subplot(132)
hist, bin_edges=np.histogram(x,bins=10)
plt.bar(bin_edges[:-1],hist,width=100)
plt.xlim(0,1024)
plt.title("X Axis Position Distribution")
plt.show()

plt.subplot(133)
hist, bin_edges=np.histogram(z,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.2)
plt.xlim(0,7)
plt.title("Z Axis distance Distribution")
plt.show()

